# Tensorflow to Qkeras to Hls4ml


In [1]:
import numpy as np

seed = 0
np.random.seed(seed)

import tensorflow as tf

tf.random.set_seed(seed)


import sys

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import dataset
import plotting
import qkeras
import keras

2024-02-02 20:31:56.265790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-02 20:31:56.385477: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-02 20:31:56.404882: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 20:31:56.719180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

# fetch dataset


In [2]:
BATCH_SIZE = 64
train_images_rgx = "../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"

train_ds, val_ds, nclasses = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE
)

2024-02-02 20:31:57.642301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-02 20:31:57.662774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2024-02-02 20:31:57.662787: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-02-02 20:31:57.663123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

# load the model


In [3]:
model_trial_id = 271

model = keras.models.load_model(
    f"../neural_network/automl/saved_models/striped_models/exp6_aug_calib/{model_trial_id}.h5",
    compile=False,
)
model.compile(
    "adam", loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"]
)

x = x_in = keras.Input(shape=(256, 256, 3))
x = keras.layers.Activation("linear")(x)
for l in model.layers[1:]:
    x = l(x)


new_model = keras.models.Model(x_in, x)
new_model.compile(
    "adam", loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"]
)

score = model.evaluate(val_ds)
score = new_model.evaluate(val_ds)
print("Accuracy: {}".format(score[1]))

1/1 [==============================] - 0s 116ms/step - loss: 0.2437 - accuracy: 0.9400
Accuracy: 0.9399999976158142


# quantize model


In [4]:
from qkeras import *
from keras.layers import Conv2D, DepthwiseConv2D, Activation, Dense, Flatten


def create_qkeras_model_from_keras_model(keras_model):
    x = x_in = keras.Input(shape=keras_model.layers[0].output.shape[1:])

    x = QActivation("quantized_relu(4,0,1)")(x)

    for l in keras_model.layers:
        if isinstance(l, Conv2D):
            qconv2d = QConv2D(
                l.filters,
                l.kernel_size,
                l.strides,
                l.padding,
                kernel_quantizer=quantized_bits(4, 0, 1),
                bias_quantizer=quantized_bits(4, 0, 1),
            )
            x = qconv2d(x)
            qconv2d.set_weights(l.get_weights())

        elif isinstance(l, DepthwiseConv2D):
            qDepthwiseConv2D = QDepthwiseConv2D(
                l.kernel_size,
                l.strides,
                l.padding,
                depthwise_quantizer=quantized_bits(4, 0, 1),
                bias_quantizer=quantized_bits(4, 0, 1),
            )
            x = qDepthwiseConv2D(x)
            qDepthwiseConv2D.set_weights(l.get_weights())

        elif isinstance(l, Activation):
            activation_name = l.get_config()["activation"]

            if activation_name == "tanh":
                x = QActivation(quantized_tanh(4, True))(x)
            elif activation_name == "relu":
                x = QActivation(quantized_relu(4, 0, use_stochastic_rounding=True))(x)
            elif activation_name == "linear":
                x = QActivation(quantized_bits(4, 0, 1, keep_negative=True))(x)
            else:
                print(f"{l.name} was not quantized")

        elif isinstance(l, Flatten):
            # necessary because the layer before Flatten doesnt have avtivation
            x = QActivation(quantized_bits(4, 0, 1, keep_negative=True))(x)
            x = Flatten()(x)

        elif isinstance(l, Dense):
            qdense = QDense(
                l.units,
                kernel_quantizer=quantized_bits(4, 0, 1),
                bias_quantizer=quantized_bits(4, 0, 1),
            )
            x = qdense(x)
            qdense.set_weights(l.get_weights())
        else:
            print(f"{l.name} layer was not quantized...")
    x = Activation("softmax")(x)

    qkeras_model = keras.models.Model(x_in, x)

    return qkeras_model


model.summary()
qkeras_model = create_qkeras_model_from_keras_model(model)
qkeras_model.compile(
    keras.optimizers.Adam(0.0005),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"],
)
qkeras_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 depthwise_conv2d_220 (Depth  (None, 85, 85, 3)        30        
 wiseConv2D)                                                     
                                                                 
 activation_759 (Activation)  (None, 85, 85, 3)        0         
                                                                 
 conv2d_810 (Conv2D)         (None, 41, 41, 24)        1824      
                                                                 
 activation_760 (Activation)  (None, 41, 41, 24)       0         
                                                                 
 depthwise_conv2d_221 (Depth  (None, 10, 10, 24)       408       
 wiseConv2D)                                                 

In [5]:
from qkeras.estimate import print_qstats

qkeras_model.evaluate(val_ds)
print_qstats(qkeras_model)

1/1 [==============================] - 1s 889ms/step - loss: 0.9573 - accuracy: 0.7400
Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.



Number of operations in model:
    q_conv2d                      : 3025800 (smult_4_4)
    q_conv2d_1                    : 96000 (smult_4_4)
    q_dense                       : 20000 (smult_4_4)
    q_depthwise_conv2d            : 195075 (smult_4_4)
    q_depthwise_conv2d_1          : 38400 (smult_4_4)

Number of operation types in model:
    smult_4_4                     : 3375275

Weight profiling:
    q_conv2d_weights               : 1800  (4-bit unit)
    q_conv2d_bias                  : 24    (4-bit unit)
    q_conv2d_1_weights             : 960   (4-bit unit)
    q_conv2d_1_bias                : 40    (4-bit unit)
    q_dense_weights                : 20000 (4-bit unit)
    q_dense_bias                   : 5     (4-bit unit)
    q_depthwise_conv2d_weights     : 81    (4-bit unit)
    q_depthwise_conv2d_bias        : 3     (4-bit unit)
    q_depthwise_conv2d_1_weights   : 9216  (4-bit unit)
    q_depthwise_conv2d_1_bias      : 24    (4-bit unit)

Weight sparsity:
... quantizing mo

# fine tune quantized model


In [6]:
qkeras_model.fit(train_ds, batch_size=64, epochs=5, validation_data=val_ds)

Epoch 1/5
10/10 [==============================] - 2s 93ms/step - loss: 0.7096 - accuracy: 0.7953 - val_loss: 0.9573 - val_accuracy: 0.7400
Epoch 2/5
10/10 [==============================] - 1s 92ms/step - loss: 0.6510 - accuracy: 0.8094 - val_loss: 0.6703 - val_accuracy: 0.8400
Epoch 3/5
10/10 [==============================] - 1s 92ms/step - loss: 0.7137 - accuracy: 0.8016 - val_loss: 0.6684 - val_accuracy: 0.8400
Epoch 4/5
10/10 [==============================] - 1s 93ms/step - loss: 0.3571 - accuracy: 0.8938 - val_loss: 0.3718 - val_accuracy: 0.9000
Epoch 5/5
10/10 [==============================] - 1s 93ms/step - loss: 0.2163 - accuracy: 0.9297 - val_loss: 0.3786 - val_accuracy: 0.9000


In [8]:
qkeras_model.save(f"saved_qmodels/{model_trial_id}.h5")

In [11]:
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)

loaded_qkeras_model = keras.models.load_model(f"saved_qmodels/{model_trial_id}.h5", co)
qkeras_model.evaluate(val_ds)

1/1 [==============================] - 0s 64ms/step - loss: 0.3786 - accuracy: 0.9000


[0.3786122798919678, 0.8999999761581421]

# convert quantized model to HLS4ml
